# Análisis exploratorio 

En ese cuaderno de realizará el análisis exploratorio relativo a `products_categories.csv`

# Índice 

1. **Importación de paquetes**
2. **Lectura de datos**
3. **Análisis**
4. **Conclusiones**

## Importación de paquete

Importamos los módulos de los que se hará uso.

In [2]:
import pandas as pd
import plotly.express as px

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Semilla para realizar experimentos reproducibles
seed = 134

## Lectura de los datos

Cargamos en memoria el conjunto de datos para comenzar a inspecccionarlo

In [3]:
categories = pd.read_csv("../Data/products_categories.csv")
categories.sample(5,random_state=seed)

,sku,cat1,cat2,cat3
10261,PR11310,Salud,Botiquín,Quemaduras y cicatrices
6818,308192,Higiene y cuidado personal,Corporal,Geles
6798,305722,Infantil,Higiene infantil,Pieles Atópicas
9012,BHC020-02,Cosmética y Belleza,Facial,Maquillajes
9238,CAU210+CAU209+CAU211,Cosmética y Belleza,Facial,Hidratantes


## Análisis

Veamos las características generales del dataset

In [4]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11159 entries, 0 to 11158
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   sku     11159 non-null  object
 1   cat1    11159 non-null  object
 2   cat2    11139 non-null  object
 3   cat3    10859 non-null  object
dtypes: object(4)
memory usage: 348.8+ KB


In [5]:
categories.describe()

,sku,cat1,cat2,cat3
count,11159,11159,11139,10859
unique,11159,8,53,271
top,00.01.10.014,Cosmética y Belleza,Facial,Antiarrugas
freq,1,3367,1789,391


Tenemos 4 campos:
- sku: identificador del producto.
- cat1: categoría principal.
- cat2: categoría secundaria.
- cat3: categoría terciaria.
  
Todos los SKU son únicos, y a primera vista existen **8** valores para **cat1**, **53** para **cat2** y **271** para **cat3**.

In [6]:
categories.isna().sum()

sku       0
cat1      0
cat2     20
cat3    300
dtype: int64

Hay valores nulos dentro de las subcategorías. El número incrementa conforme se baja de nivel

In [7]:
categories["cat1"].value_counts()

Cosmética y Belleza           3367
Higiene y cuidado personal    2994
Salud                         2852
Infantil                      1304
Nutrición                      618
Veterinaria                     20
salud                            3
nutrición                        1
Name: cat1, dtype: int64

La categoría predominante es *Cosmética y Belleza*, mientras que otras categorías carecen de un gran número de instancias. Algo de lo que nos percatamos es que los nombres no están estandarizados y de ahí que *Salud* y *Nutrición* las contabiliza de forma diferentes.

In [8]:
categories["cat2"].value_counts()

Facial                                      1789
Corporal                                     905
Cabello                                      855
Dental                                       738
Vitaminas y Minerales                        644
Botiquín                                     474
Sol                                          422
Íntima                                       298
Higiene infantil                             286
Ojos                                         274
Pérdida de peso                              265
Huesos, articulaciones y músculos            246
Labial                                       223
Biberones                                    206
Trastornos digestivos                        196
Uñas                                         194
Accesorios de alimentación                   187
Alimentos                                    182
Aromaterapia                                 181
Perfumería                                   166
Problemas cardiovasc

In [9]:
categories["cat3"].value_counts()

Antiarrugas                      391
Hidratantes                      345
Desmaquillantes y limpiadores    313
Hidratación                      288
Protectores solares              283
                                ... 
 Vitaminas                         1
nerviosismo y estrés               1
Pieles atópicas                    1
Sujetadores Lactancia              1
Gel desinfectante                  1
Name: cat3, Length: 271, dtype: int64

Las subcategorías dos y tres parece ser que tienen más opciones entre las que elegir, algo que tiene sentido, ya que están a un nivel jerárquico inferior. Mientras que al respecto las minúsculas y mayúsculas ocurre algo parecido que antes, así que arreglaremos estas erratas normalizando los nombres de las categorías.

In [10]:
categories.loc[:, ["cat1", "cat2", "cat3"]] = categories[["cat1", "cat2", "cat3"]].apply(lambda r: [str(n).strip().capitalize() for n in r], axis=1, result_type="broadcast")
categories.describe()

,sku,cat1,cat2,cat3
count,11159,11159,11159,11159
unique,11159,6,50,250
top,00.01.10.014,Cosmética y belleza,Facial,Antiarrugas
freq,1,3367,1789,391


Ahora solo tenemos **6** valores para **cat1**, **50** para **cat2** y **250** para **cat3**.

Observemos la relación jerárquica entre las categorías

In [11]:
counts = categories.value_counts(subset=["cat1", "cat2", "cat3"]).reset_index(name="count")
counts.sample(5,random_state=seed)

,cat1,cat2,cat3,count
159,Higiene y cuidado personal,Cabello,Antigrasa,15
39,Salud,Problemas cardiovasculares y circulación,Colesterol,85
224,Infantil,Lactancia,Cojin de lactancia,5
219,Higiene y cuidado personal,Dental,Irrigadores bucales,5
2,Cosmética y belleza,Corporal,Hidratación,288


In [12]:
fig = px.sunburst(counts, path=["cat1", "cat2", "cat3"], values="count", height=1300)
fig.show()

Notamos que algunas productos de ciertas categorías primarias y secundarias admiten no tener categoría terciaria, como es el caso de **Vitaminas y minerales**, o **Descanso y relajación** en **Salud**. También hay una categoría primaria (**Veterinaria**) que a parte de no tener apenas productos, tampoco tiene categorías secundarias asociadas.

Por otro lado, se ve como algunas de las categorías principales están más *"seccionadas"* que otras y lo mismo ocurre con las categorías secundarias. Seccionar más una categoría puede implicar una mejor clasificación, pero puede suponer que haya menos productos que cumplan toda una taxonomía tan estricta y deban ser dejados de lado o crear otra categoría específica para ellos. Esto puede suponer crear categorías *"vacías"*

Observemos ahora las categorías individualmente:

In [13]:
counts_1 = counts.groupby("cat1", as_index=False).agg({"count":"sum"})
fig = px.pie(counts_1, values="count", names="cat1")
fig.show()

Vemos que el *"pastel"* se lo reparten tres categorías principales, tal y como veníamos viendo de antes.

In [14]:
fig = px.bar(categories["cat2"].value_counts())
fig.show()

En esta subcategoría observamos que *Facial* sobresale a primera vista, mientras que otras no son tan comunes. Existe una gran descompensación entre *"clases"*

In [15]:
fig = px.bar(categories["cat3"].value_counts(), orientation="h")
fig.show()

El mismo efecto de descompensación se intensifica en el tercer nivel de jerarquía.

## Conclusiones 

De aquí sacamos varias conclusiones:

- Existen valores nulos dentro de las subcategorías
- Los nombres de las categorías no están regularizados
- Algunas subcategorías están más *segmentadas* que otras
- La distribución de categorías no es uniforme